# Building a Local MCP Client with LlamaIndex

This Jupyter notebook walks you through creating a **local MCP (Model Context Protocol) client** that can chat with a database through tools exposed by an MCP server—completely on your machine. Follow the cells in order for a smooth, self‑contained tutorial.

In [ ]:
%pip install llama-index

import nest_asyncio
nest_asyncio.apply()

from llama_index.llms.ollama import Ollama
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.workflow import Context
from llama_index.core.agent.workflow import ToolCallResult, ToolCall

## 2  Setup a local LLM

In [ ]:
llm = Ollama(model="llama3.2", request_timeout=120.0)
Settings.llm = llm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 1.9 MB/s eta 0:00:001.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.1 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 2.0 MB/s eta 0:00:001.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 2.0 MB/s eta 0:00:002.1 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 kB 2.0 MB/s eta 0:00:00? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 2.0 MB/s eta 0:00:000m eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.3 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 2.4 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.5/585.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.0 MB/s eta 0:00:00 MB/s eta 0:00:01
  Attempting uninstall: setuptools5;237m╺━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 3  Initialize the MCP client and build the agent
Point the client at your local MCP server’s **SSE endpoint** (default shown below), and list the available tools.

In [ ]:
%pip install llama-index

from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tool = McpToolSpec(client=mcp_client)

INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.2/755.2 kB 1.6 MB/s eta 0:00:001.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 2.6 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26/26 [llama-index]━━━━ 20/26 [llama-index-cli]
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'llama_index.tools'

In [4]:
tools = await mcp_tool.to_tool_list_async()
for tool in tools:
    print(tool.metadata.name, tool.metadata.description)

NameError: name 'mcp_tool' is not defined

## 3  Define the system prompt
This prompt steers the LLM when it needs to decide how and when to call tools.

In [6]:
SYSTEM_PROMPT = """\
You are an AI assistant for Tool Calling.

Before you help a user, you need to work with tools to interact with Our Database
"""

## 4  Helper function: `get_agent()`
Creates a `FunctionAgent` wired up with the MCP tool list and your chosen LLM.

In [7]:
from llama_index.tools.mcp import McpToolSpec
from llama_index.core.agent.workflow import FunctionAgent

async def get_agent(tools: McpToolSpec):
    tools = await tools.to_tool_list_async()
    agent = FunctionAgent(
        name="Agent",
        description="An agent that can work with Our Database software.",
        tools=tools,
        llm=OpenAI(model="gpt-4"),
        system_prompt=SYSTEM_PROMPT,
    )
    return agent

## 5  Helper function: `handle_user_message()`
Streams intermediate tool calls (for transparency) and returns the final response.

In [8]:
from llama_index.core.agent.workflow import (
    FunctionAgent, 
    ToolCallResult, 
    ToolCall)

from llama_index.core.workflow import Context

async def handle_user_message(
    message_content: str,
    agent: FunctionAgent,
    agent_context: Context,
    verbose: bool = False,
):
    handler = agent.run(message_content, ctx=agent_context)
    async for event in handler.stream_events():
        if verbose and type(event) == ToolCall:
            print(f"Calling tool {event.tool_name} with kwargs {event.tool_kwargs}")
        elif verbose and type(event) == ToolCallResult:
            print(f"Tool {event.tool_name} returned {event.tool_output}")

    response = await handler
    return str(response)

## 6  Initialize the MCP client and build the agent
Point the client at your local MCP server’s **SSE endpoint** (default shown below), build the agent, and setup agent context.

In [9]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec


mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tool = McpToolSpec(client=mcp_client)

# get the agent
agent = await get_agent(mcp_tool)

# create the agent context
agent_context = Context(agent)

In [ ]:
# Run the agent!
while True:
    user_input = input("Enter your message: ")
    if user_input == "exit":
        break
    print("User: ", user_input)
    response = await handle_user_message(user_input, agent, agent_context, verbose=True)
    print("Agent: ", response)

User:  Add to the db: Rafael Nadal whose age is 39 and is a tennis player
Calling tool add_data with kwargs {'query': "INSERT INTO people (name, age, profession) VALUES ('Rafael Nadal', 39, 'Tennis Player')"}
Tool add_data returned meta=None content=[TextContent(type='text', text='true', annotations=None)] isError=False
Agent:  The data has been added successfully.
User:  fetch data
Calling tool read_data with kwargs {'query': 'SELECT * FROM people'}
Tool read_data returned meta=None content=[TextContent(type='text', text='1', annotations=None), TextContent(type='text', text='Rafael Nadal', annotations=None), TextContent(type='text', text='39', annotations=None), TextContent(type='text', text='Tennis Player', annotations=None)] isError=False
Agent:  Here is the data from the database:

1. ID: 1
   Name: Rafael Nadal
   Age: 39
   Profession: Tennis Player
User:  add to the db: Roger federer whose age is 42 and is a tennis player
Calling tool add_data with kwargs {'query': "INSERT INTO 